In [2]:
import os
import json
import re
import pandas as pd

# Pfad zum Hauptverzeichnis mit den Unterverzeichnissen
BASE_DIR = "data/liboqs"

def extract_model_name(directory_name):
    """Extrahiert den Modelnamen aus dem Verzeichnisnamen."""
    match = re.match(r"testresult_liboqs_(.*?)_\d{4}\.\d{2}\.\d{2}_\d{2}\.\d{2}", directory_name)
    return match.group(1) if match else directory_name

# Lade die JSON-Datei
def load_kem_memory_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

# Extrahiere relevante Daten aus der JSON-Datei
def parse_kem_memory_data(data, model_name):
    records = []
    for algorithm, details in data.items():
        if algorithm in ["cpuinfo", "config"]:
            continue  # Überspringe Meta-Daten

        keygen = details.get("keygen", {})
        encaps = details.get("encaps", {})
        decaps = details.get("decaps", {})

        records.append({
            "Model": model_name,
            "Algorithmus": algorithm,
            "maxHeap_key": keygen.get("maxHeap", 0),
            "maxHeap_enc": encaps.get("maxHeap", 0),
            "maxHeap_dec": decaps.get("maxHeap", 0),
            "maxStack_key": keygen.get("maxStack", 0),
            "maxStack_enc": encaps.get("maxStack", 0),
            "maxStack_dec": decaps.get("maxStack", 0)
        })
    
    return records

# Hauptfunktion zum Laden und Verarbeiten der Daten
def main():
    all_records = []
    
    for directory in os.listdir(BASE_DIR):
        full_path = os.path.join(BASE_DIR, directory)
        if os.path.isdir(full_path):
            json_file = os.path.join(full_path, "test_kem_mem.json")
            if os.path.isfile(json_file):
                model_name = extract_model_name(directory)
                data = load_kem_memory_data(json_file)
                recs = parse_kem_memory_data(data, model_name)
                all_records.extend(recs)
    
    # DataFrame erstellen
    df = pd.DataFrame(all_records)
    df = df.sort_values(by=["Algorithmus", "Model"])
    
    # Speichere als CSV
    df.to_csv("output/csv/liboqs_mem_kem.csv", index=False)
    
    # Anzeige im Jupyter Notebook
    from IPython.display import display
    display(df)
    
    return df

# Starte die Verarbeitung
df = main()


,Model,Algorithmus,maxHeap_key,maxHeap_enc,maxHeap_dec,maxStack_key,maxStack_enc,maxStack_dec
52,Pi3b,BIKE-L1,10940,12545,12577,35760,26016,79392
81,Pi4b,BIKE-L1,10940,12545,12577,35760,26016,79392
136,PiZero2,BIKE-L1,10940,12545,12577,35760,26016,79392
53,Pi3b,BIKE-L3,17364,20511,20543,69552,50816,156720
82,Pi4b,BIKE-L3,17364,20511,20543,69552,50816,156720
...,...,...,...,...,...,...,...,...
74,Pi3b,sntrup761,11665,12736,8200,8720,6768,11760
103,Pi4b,sntrup761,11665,12736,8200,8720,6768,11760
129,PiBanana,sntrup761,307841,312677,308944,5848,6040,7568
181,PiZero,sntrup761,11465,12536,8160,8392,6024,11640
